In [6]:
# ==============================================================================
# ELLIPTIC CURVE GROUP DATA COLLECTOR (WITH DELTA) - REVISED
# Collects data on all five groups for elliptic curves in Cremona database
# Outputs: CSV file
# ==============================================================================

import csv
from datetime import datetime

# ==============================================================================
# CONFIGURATION - EDIT THESE SETTINGS
# ==============================================================================

# Conductor bound
conductor_bound = 54

# Output directory (use empty string "" for current directory)
output_dir = ""

# Output filenames (auto-generated with timestamp, or customize below)
use_timestamp = True  # Set to False to use custom filenames

if use_timestamp:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"elliptic_curve_groups_1_to_{conductor_bound}_{timestamp}.csv"
else:
    # Custom filenames (edit these if use_timestamp = False)
    csv_filename = "my_curve_data.csv"

# Add directory path to filename
if output_dir:
    csv_filename = output_dir + csv_filename

# Print configuration
print("=" * 70)
print("ELLIPTIC CURVE GROUP DATA COLLECTOR (WITH DELTA)")
print("=" * 70)
print(f"Conductor bound: {conductor_bound}")
print(f"Output directory: {output_dir if output_dir else '(current directory)'}")
print(f"CSV filename: {csv_filename}")
print("=" * 70)
print()

# ==============================================================================
# 1. SETUP (Column Headers)
# ==============================================================================

# CSV column headers - REVISED per user request
headers = [
    # Identification
    "label",                                    # A
    "conductor",                                # B
    # OMITTED: conductor_factored               # (was C)
    # OMITTED: conductor_is_prime               # (was D)
    # OMITTED: conductor_num_prime_factors      # (was E)
    # OMITTED: conductor_num_prime_factors_with_mult  # (was F)
    "weight",                                   # G

    # Group 1: Torsion subgroup
    "torsion_order",                            # H
    "torsion_structure",                        # I

    # Group 2: Rank and generators
    "rank",                                     # J
    "rank_is_certain",                          # K
    "num_generators",                           # L
    "generator_heights",                        # M
    "regulator",                                # N

    # Group 3: Galois representation (relabeled)
    "galois_rep_surjective_all_primes",        # O (was galois_rep_surjective)
    "galois_rep_exceptional_primes",           # P (was galois_rep_non_surjective_primes)
    "galois_image_type",                        # Q

    # Group 4: Gamma_0(N)
    "gamma0_index_in_SL2Z",                     # R
    "X0N_genus",                                # S
    "X0N_num_cusps",                            # T

    # Group 5: Modular form
    "is_CM",                                    # U
    "CM_discriminant",                          # V
    "root_number",                              # W
    # OMITTED: ap_coefficients_first_10_primes  # (was X)
    # OMITTED: oscillates_all_n                 # (was Y)
    # OMITTED: oscillates_primes                # (was Z)
    # OMITTED: notes
]

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================

def torsion_structure_string(E):
    """Return torsion subgroup structure as string"""
    T = E.torsion_subgroup()
    inv = T.invariants()
    if len(inv) == 0:
        return "trivial"
    else:
        return " x ".join([f"Z/{n}Z" for n in inv])

def generator_heights_string(E):
    """Return list of generator canonical heights - FIXED BUG"""
    try:
        gens = E.gens()
        if len(gens) == 0:
            return "none"
        # FIX: height() is a method of the point, not the curve
        heights = [float(g.height()) for g in gens]
        return "|".join([f"{h:.4f}" for h in heights])
    except Exception as ex:
        return f"error: {ex}"

def galois_image_type(E):
    """Classify Galois image type"""
    try:
        rho = E.galois_representation()
        if E.has_cm():
            return "CM"
        non_surj = rho.non_surjective()
        if len(non_surj) == 0:
            return "surjective"
        else:
            return f"non-surjective at {list(non_surj)}"
    except Exception as ex:
        return f"error: {ex}"

def X0N_data(N):
    """Compute data about X_0(N)"""
    try:
        G = Gamma0(N)
        idx = G.index()
        genus = G.genus()
        cusps = G.ncusps()
        return idx, genus, cusps
    except Exception as ex:
        return "error", "error", "error"

# ==============================================================================
# 3. CREATE DELTA ROW (RAMANUJAN'S DELTA FUNCTION)
# ==============================================================================

def create_delta_row():
    """Create special row for Ramanujan's Delta function"""
    
    row = {
        # Identification
        "label": "Delta",
        "conductor": 1,
        "weight": 12,
        
        # Group 1: Torsion subgroup - N.A. for modular forms
        "torsion_order": "N.A.",
        "torsion_structure": "N.A.",
        
        # Group 2: Rank and generators - N.A. for modular forms
        "rank": "N.A.",
        "rank_is_certain": "N.A.",
        "num_generators": "N.A.",
        "generator_heights": "N.A.",
        "regulator": "N.A.",
        
        # Group 3: Galois representation - EXISTS for Delta
        "galois_rep_surjective_all_primes": True,
        "galois_rep_exceptional_primes": "none",
        "galois_image_type": "surjective (weight 12)",
        
        # Group 4: Gamma_0(N) - Delta lives on SL_2(Z) = Gamma_0(1)
        "gamma0_index_in_SL2Z": 1,
        "X0N_genus": 0,
        "X0N_num_cusps": 1,
        
        # Group 5: Modular form properties
        "is_CM": False,
        "CM_discriminant": 0,
        "root_number": 1,
    }
    
    return row

# ==============================================================================
# 4. MAIN LOOP: COLLECT DATA FOR ALL CURVES
# ==============================================================================

all_rows = []
curve_count = 0
error_count = 0

# First, add Delta
print("Adding special entry for Ramanujan's Delta function...")
delta_row = create_delta_row()
all_rows.append(delta_row)
print(f"  Delta         conductor=1     weight=12")
print()

print(f"Collecting data for elliptic curves with conductor <= {conductor_bound}...")
print()

for N in range(1, conductor_bound + 1):

    # Get all curves of this conductor from Cremona database
    try:
        curves = cremona_curves([N])
    except Exception:
        continue

    for E_cremona in curves:

        label = E_cremona.label()

        try:
            # Load curve in SageMath
            E = EllipticCurve(label)

            # --- Identification ---
            conductor = int(E.conductor())
            weight = 2  # All elliptic curves have weight 2

            # --- Group 1: Torsion ---
            torsion_order = int(E.torsion_order())
            torsion_structure = torsion_structure_string(E)

            # --- Group 2: Rank ---
            try:
                rank = int(E.rank())
                rank_certain = True
            except Exception:
                rank = -1
                rank_certain = False

            try:
                gens = E.gens()
                num_gens = len(gens)
            except Exception:
                num_gens = -1

            gen_heights = generator_heights_string(E)

            try:
                reg = float(E.regulator())
            except Exception:
                reg = -1.0

            # --- Group 3: Galois representation ---
            try:
                rho = E.galois_representation()
                non_surj = list(rho.non_surjective())
                gal_surjective = (len(non_surj) == 0)
                gal_exceptional_primes = "|".join(
                    [str(p) for p in non_surj]) if non_surj else "none"
            except Exception as ex:
                gal_surjective = "error"
                gal_exceptional_primes = str(ex)

            gal_image = galois_image_type(E)

            # --- Group 4: Gamma_0(N) ---
            idx, genus, cusps = X0N_data(conductor)

            # --- Group 5: Modular form ---
            try:
                is_cm = E.has_cm()
                cm_disc = int(E.cm_discriminant()) if is_cm else 0
            except Exception:
                is_cm = "error"
                cm_disc = "error"

            try:
                root_number = int(E.root_number())
            except Exception:
                root_number = "error"

            # --- Assemble row ---
            row = {
                "label": label,
                "conductor": conductor,
                "weight": weight,
                "torsion_order": torsion_order,
                "torsion_structure": torsion_structure,
                "rank": rank,
                "rank_is_certain": rank_certain,
                "num_generators": num_gens,
                "generator_heights": gen_heights,
                "regulator": reg,
                "galois_rep_surjective_all_primes": gal_surjective,
                "galois_rep_exceptional_primes": gal_exceptional_primes,
                "galois_image_type": gal_image,
                "gamma0_index_in_SL2Z": idx,
                "X0N_genus": genus,
                "X0N_num_cusps": cusps,
                "is_CM": is_cm,
                "CM_discriminant": cm_disc,
                "root_number": root_number,
            }

            all_rows.append(row)
            curve_count += 1

            # Print progress
            print(f"  {label:12s}  conductor={conductor:4d}  "
                  f"rank={rank}  torsion={torsion_structure:20s}  "
                  f"CM={is_cm}  root_number={root_number}")

        except Exception as ex:
            error_count += 1
            print(f"  ERROR on {label}: {ex}")
            continue

# ==============================================================================
# 5. SAVE TO CSV
# ==============================================================================

print()
print("=" * 70)
print(f"Data collection complete.")
print(f"  Delta: 1 entry")
print(f"  Elliptic curves processed: {curve_count}")
print(f"  Total entries: {len(all_rows)}")
print(f"  Errors: {error_count}")
print()
print(f"Saving to CSV: {csv_filename}")

with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(all_rows)

print(f"CSV saved successfully!")
print()

# ==============================================================================
# 6. SUMMARY STATISTICS
# ==============================================================================

print("=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

# Separate Delta from elliptic curves for statistics
elliptic_rows = [r for r in all_rows if r["label"] != "Delta"]

ranks = [r["rank"] for r in elliptic_rows if isinstance(r["rank"], int) and r["rank"] >= 0]
print(f"Total entries: {len(all_rows)} (1 Delta + {len(elliptic_rows)} elliptic curves)")
print()
print("ELLIPTIC CURVES:")
print(f"  Rank 0: {sum(1 for r in ranks if r == 0)}")
print(f"  Rank 1: {sum(1 for r in ranks if r == 1)}")
print(f"  Rank 2: {sum(1 for r in ranks if r == 2)}")
print(f"  Rank >= 3: {sum(1 for r in ranks if r >= 3)}")
print()

torsion_counts = {}
for row in elliptic_rows:
    t = row["torsion_structure"]
    torsion_counts[t] = torsion_counts.get(t, 0) + 1
print("Torsion subgroup distribution:")
for t, count in sorted(torsion_counts.items(),
                        key=lambda x: -x[1])[:10]:  # Show top 10
    print(f"  {t}: {count}")
print()

cm_count = sum(1 for r in elliptic_rows if r["is_CM"] is True)
print(f"CM curves: {cm_count}")
print(f"Non-CM curves: {len(elliptic_rows) - cm_count}")
print()

root_pos = sum(1 for r in elliptic_rows if r["root_number"] == 1)
root_neg = sum(1 for r in elliptic_rows if r["root_number"] == -1)
print(f"Root number +1: {root_pos}")
print(f"Root number -1: {root_neg}")

print()
print("=" * 70)
print(f"CSV saved as: {csv_filename}")
print("=" * 70)

ELLIPTIC CURVE GROUP DATA COLLECTOR (WITH DELTA)
Conductor bound: 54
Output directory: (current directory)
CSV filename: elliptic_curve_groups_1_to_54_20260224_175619.csv

Adding special entry for Ramanujan's Delta function...
  Delta         conductor=1     weight=12


  11a1          conductor=  11  rank=0  torsion=Z/5Z                  CM=False  root_number=1
  11a2          conductor=  11  rank=0  torsion=trivial               CM=False  root_number=1
  11a3          conductor=  11  rank=0  torsion=Z/5Z                  CM=False  root_number=1
  14a1          conductor=  14  rank=0  torsion=Z/6Z                  CM=False  root_number=1
  14a2          conductor=  14  rank=0  torsion=Z/6Z                  CM=False  root_number=1
  14a3          conductor=  14  rank=0  torsion=Z/2Z                  CM=False  root_number=1
  14a4          conductor=  14  rank=0  torsion=Z/6Z                  CM=False  root_number=1
  14a5          conductor=  14  rank=0  torsion=Z/2Z                  C